# How to grab key-value pairs from a local file

If you haven’t already, create a Lazarus Forms account (link to the get started guide) and grab you **authKey** and **orgId**.
___

The only package you will need to import is the requests package

In [ ]:
import requests

For this example, we will be using an example form that is attached to this notebook, though you can select another image or pdf.

In [ ]:
form_to_analyze = open('./Example Form.png', 'rb')

Once you have a pdf or image that you want to analyze, we can start making a POST request to the url below:

In [ ]:
url = "https://api.lazarusforms.com/api/forms/generic"

As stated in the docs, we are going to need to pass the **authKey** and **orgId** in the headers.

**Make sure to add in your credentials here.**

In [ ]:
headers = {
  'orgId': 'your orgID here',
  'authKey': 'your authKey here'
}

We will also have to pass **form_to_analyze** (as part of a multipart form).

In [ ]:
files=[
  ('file',('file_name', form_to_analyze, 'application/octet-stream'))
]

Now we have all we need to run the POST request!

In [ ]:
response = requests.post(url, headers=headers, files=files)

If all went well, you should have recieved a JSON response.

There is a chance you run into an error here so it is best to use a try-except here.

In [ ]:
try:
    data_to_use = response.json()
except:
    print("There was an error")
    print(response)

While having a giant JSON is nice to look at, in this task, we are only interested in the 'keyValuePairs', which is an array.

In [ ]:
key_values_pairs = data_to_use['keyValuePairs']

Now we need to parse through the pairs and grab the content of each key and value.

One thing to note is that some pairs may have a missing value if the model did not detect it or the form field was left blank, in this case we can avoid grabing that pair. 

In [ ]:
key_values = []
for pair in key_values_pairs:
    try:
        key_values.append({
            'key' : pair['key']['content'],
            'value' : pair['value']['content']
        })
    except:
        pass

At this point, we now have a list of filled in form fields.

In [ ]:
print(key_values)

As mentioned earlier, there may be a case where some of the fields were left blank or not detected, it would be useful to collect those incomplete key-values a present them to a reviewer to confirm and make corrections.

In [ ]:
key_values = []
missing_values = []
for pair in key_values_pairs:
    try:
        key_values.append({
            'key' : pair['key']['content'],
            'value' : pair['value']['content']
        })
    except:
        print('Missing: ' + pair['key']['content'])
        missing_values.append({
            'key' : pair['key']['content']
        })